In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
import json
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer


ruta = "F:/nuevo_conocimiento/CODIGO DE TESIS (MUY IMPORTANTE)/Experimentos_con_modelos_basicos/dataset_preprocesado/dataset_abstract_preprocesado.csv"

datos = pd.read_csv(ruta)
datos

,title,abstract,categorical,categoria,cat_one_hot,abstract_limpio,abstract_preprocesado
0,Electromagnetic neutrinos in laboratory experi...,An overview of neutrino electromagnetic proper...,astrophysics,humano,"[1, 0, 0, 0, 0]",An overview of neutrino electromagnetic proper...,"['overview', 'neutrino', 'electromagnetic', 'p..."
1,Infrared complex refractive index of astrophys...,In dense and cold regions of the interstellar ...,astrophysics,humano,"[1, 0, 0, 0, 0]",In dense and cold regions of the interstellar ...,"['dense', 'cold', 'region', 'interstellar', 'm..."
2,Gamma-ray lines in modern astrophysics,Gamma-ray astronomy provides a direct window i...,astrophysics,humano,"[1, 0, 0, 0, 0]",Gamma-ray astronomy provides a direct window i...,"['gamma-ray', 'astronomy', 'provides', 'direct..."
3,From Coulomb excitation cross sections to non-...,Coulomb and nuclear dissociation of $^{17}$Ne ...,astrophysics,humano,"[1, 0, 0, 0, 0]",Coulomb and nuclear dissociation of $^{17}$Ne ...,"['coulomb', 'nuclear', 'dissociation', 'ne', '..."
4,Astrophysical Black Holes: A Compact Pedagogic...,Black holes are among the most extreme objects...,astrophysics,humano,"[1, 0, 0, 0, 0]",Black holes are among the most extreme objects...,"['black', 'hole', 'among', 'extreme', 'object'..."
...,...,...,...,...,...,...,...
7745,Superconducting LaAlO3/SrTiO3 Nanowires,Here's a rewritten version of the text using m...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nWe've achieved a groundbreaking breakthrou...,"[""'ve"", 'achieved', 'groundbreaking', 'breakth..."
7746,Coupling a single Nitrogen-Vacancy center to a...,Here's a rewritten version of the text in my o...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nWe propose a groundbreaking idea: connecti...,"['propose', 'groundbreaking', 'idea', 'connect..."
7747,Superconducting and Anti-Ferromagnetic Phases ...,Here's a rewritten version of the text using m...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nThis study establishes a connection betwee...,"['study', 'establishes', 'connection', 'two', ..."
7748,New Superconductivity Dome in LaFeAsO$_{1-x}$F...,Here's the rewritten text:\n\nA fascinating ph...,superconductivity,llama3,"[0, 0, 0, 0, 1]",Here's the rewritten text:\n\nA fascinating ph...,"[""'s"", 'rewritten', 'text', 'fascinating', 'ph..."


In [4]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llava",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [5]:
textos = datos['abstract_limpio'].tolist()
etiquetas = datos['cat_one_hot'].tolist()

pass_embedding = ollama_embedding.get_text_embedding_batch(
   textos, show_progress=True
)

print(len(pass_embedding))

Generating embeddings:   0%|          | 0/7750 [00:00<?, ?it/s]

7750


In [6]:
df = pd.DataFrame(pass_embedding)
df

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,3.162146,-0.903273,-1.736204,1.288543,6.582812,-8.629716,-0.022527,-2.852088,2.878001,0.542700,...,-3.817014,-6.237505,1.750852,7.205899,6.631708,15.516543,9.016453,-8.517712,0.583288,4.642850
1,4.880323,1.025220,-0.284510,5.588247,3.900945,-11.105122,0.759113,-2.717180,5.075887,-1.579835,...,1.239750,-9.479925,2.202733,-2.352997,5.044775,3.540188,4.750847,-2.541020,-0.080359,3.478499
2,3.341349,5.170041,-4.253198,-2.556113,5.206687,-4.903310,3.145244,0.424181,1.498327,-3.303890,...,0.487464,-5.666276,2.491357,6.024241,-1.802036,9.689981,4.174418,-4.244342,1.844876,5.941211
3,4.633794,0.989607,3.441152,1.156447,2.409604,-9.179559,-0.118940,-2.259145,5.621871,-6.830520,...,-2.309913,-6.632195,0.273196,12.989382,4.638300,14.461243,0.263578,-6.580178,-1.954032,1.302500
4,5.374534,2.544695,-5.302870,-4.133938,-0.853640,-7.507550,3.036594,3.267350,4.360455,-0.806205,...,-4.051067,-7.307005,3.909728,2.109147,4.873223,13.659921,6.008616,-0.105482,4.884347,1.653572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,2.732642,4.737146,-0.184927,5.996357,1.542229,-7.799791,4.737806,-6.243899,1.172513,-0.161979,...,-3.146632,-2.625686,0.077290,4.509640,-0.138913,1.794293,5.252464,5.543174,3.986314,0.867304
7746,1.366801,1.692726,1.526221,1.491258,1.801466,-9.366475,-0.789858,0.529328,0.171018,-7.002450,...,0.284211,-7.208097,1.823583,11.192524,-1.947234,10.314806,4.044218,-3.058365,5.256784,3.087132
7747,3.245125,2.384629,0.095048,1.510087,4.234876,-5.507205,-3.610924,-6.655462,2.514819,-3.616803,...,-4.398721,-3.510185,0.743546,6.826065,3.710402,11.279869,4.678563,2.706820,5.159875,1.508390
7748,4.851994,0.563872,2.451720,4.906705,-0.299685,-6.939936,-0.790690,-3.202202,1.497667,-3.001094,...,-2.443645,-4.739741,1.764148,8.002017,-1.157072,10.755685,9.030517,4.836179,2.876370,2.784364


In [7]:
df['cat_one_hot'] = etiquetas
df.to_csv('llava_embeddings.csv', index=False)  # Para CSV